#ETL com python e pandas

O que é ETL

ETL vem do nome Extract-Transform-Load (em português: Extrair-Transformar-Carregar). Chama-se de ETL todo processo onde se extrai dados geralmente de fontes brutas (Banco de dados de aplicações, APIs de aplicações web, dados provenientes de Web-Scraping etc) e os trata para posteriormente serem usados tanto por seres humanos (análises e criação de dashboards) quando por máquinas (Machine Learning). Neste exemplo, iremos ler dados de relatórios extraídos em formato de planilha e os trataremos para realizar possíveis análises

Nossa base

Nessa aula iremos realizar uma análise da base de clientes de um e-commerce localizado na cidade de Franca

## Importação das bibliotecas

In [7]:
# Autenticação para ler arquivos do Google Drive
from google.colab import drive, auth

# Bibliotecas para realizar tratativas de dados
import pandas as pd
import numpy as np
from datetime import datetime, date

# GLOB: facilita leitura de diretórios
from glob import glob

# Bibliotecas de visualização de dados
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Montagem do drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Tratativa da base de cliente

###Leitura de arquivos

A biblioteca glob permite lermos todos os arquivos de uma pasta. Com isto, iteramos cada um dos arquivos, lendo-os e consolidando-os em uma lista de dataframes, onde podemos usar a função concat para criar um dataframe único

In [8]:
arquivos_clientes = glob("/content/drive/MyDrive/Projetos/Aula pandas/Relatórios de cliente anonimizados/*.xlsx")
dataframe_list_clientes = []
for arquivo in arquivos_clientes:
    dataframe_temp_clientes = pd.read_excel(io=arquivo)
    dataframe_list_clientes.append(dataframe_temp_clientes)
dataframe_clientes_raw = pd.concat(dataframe_list_clientes)

KeyboardInterrupt: 

### Tratativas iniciais e seleções de coluna

Selecionando apenas colunas que serão usadas e as renomeando para padronizar

Veja a sintaxe para selecionar colunas usando dois colchetes.

In [ ]:
dataframe_clientes = dataframe_clientes[[
    "id",
    "CPF",
    "Cidade",
    "Estado",
    "Sexo",
    "Data Cadastro",
    "Hora Cadastro",
    "nascimento"
]]

dataframe_clientes = dataframe_clientes.rename(columns={
    "id": "id_cliente",
    "Cidade": "cidade",
    "Estado": "estado",
    "Sexo": "sexo",
    "nascimento": "data_nascimento",
    "CPF": "cpf",
})

Explorando os primeiros resultados

A função .head() (do inglês 'cabeça') permite exibir os N primeiros registros de um dataframe, com o valor de N padronizado em 5.

In [ ]:
dataframe_clientes.head()

O atributo shape permite saber a quantidade de linhas e colunas do nosso dataframe

In [ ]:
dataframe_clientes.shape

A função count chamada diretamente no dataframe irá retornar a contagem de valores não nulos em cada coluna.

In [ ]:
dataframe_clientes.count()

Perceba que temos 7 registros que estão com alguns campos em branco

O atributo dtypes mostra o tipo de cada coluna. Isto é importante para checarmos se as colunas estão em formato numérico, string, data, booleano, etc. o valor 'object' geralmente significa que a coluna é do tipo string, porém pode representar um tipo não mapeado pelo pandas

In [ ]:
dataframe_clientes.head()

### Tratativas avançadas

Corrigindo tipo das colunas de data

Ao se trabalhar com datas, é importante que o formato da coluna seja do tipo datetime. Isso nos vai permitir fazer comparações entre datas sem erros, além de permitir operações envolvendo datas.

In [ ]:
dataframe_clientes["data_cadastro"] = dataframe_clientes["Data Cadastro"] + dataframe_clientes['Hora Cadastro']
dataframe_clientes.head()

In [ ]:
dataframe_clientes["data_cadastro"] = pd.to_datetime(dataframe_clientes["data_cadastro"], format="%d/%m/%Y%H:%M:%S").dt.floor('s')
dataframe_clientes["data_nascimento"] = pd.to_datetime(dataframe_clientes["data_nascimento"], format="%Y-%m-%d")
dataframe_clientes.head()

In [ ]:
dataframe_clientes.dtypes

Coluna de Região do Brasil. Realizando um "PROCV" no pandas

Nesta etapa, instanciei um dicionário fazendo um De-para de estado para região que o estado pertence.

In [ ]:
de_para_regioes = {"AC": "Norte",
"AL": "Nordeste",
"AP": "Norte",
"AM": "Norte",
"BA": "Nordeste",
"CE": "Nordeste",
"DF": "Centro-oeste",
"ES": "Sudeste",
"GO": "Centro-oeste",
"MA": "Nordeste",
"MT": "Centro-oeste",
"MS": "Centro-oeste",
"MG": "Sudeste",
"PA": "Norte",
"PB": "Nordeste",
"PR": "Sul",
"PE": "Nordeste",
"PI": "Nordeste",
"RJ": "Sudeste",
"RN": "Nordeste",
"RS": "Sul",
"RO": "Norte",
"RR": "Norte",
"SC": "Sul",
"SP": "Sudeste",
"SE": "Nordeste",
"TO": "Norte"}
dataframe_clientes["regiao"] = dataframe_clientes["estado"].map(de_para_regioes)

Coluna de idade: calculando qual era a idade do cliente no dia 01/09/2023

Aqui usamos a coluna de data de nascimento já parametrizada para fazer um cálculo da idade dos clientes

In [ ]:
dataframe_clientes["idade"] = round((datetime(2023,9,1,0,0,0) - dataframe_clientes['data_nascimento']).dt.days / 365.25)

Coluna da faixa etária. Usando a função pd.cut para simplificar o processo de clusterização

Iremos trabalhar com faixas etárias e não com a idade cheia, para que façamos análises com grupos maiores.

In [ ]:
dataframe_clientes["faixa_etaria"] = pd.cut(x=dataframe_clientes["idade"],
                                            bins=[-1,14,17,24,35,45,55,65,999],
                                            labels=["0 a 14 anos", "15 a 17 anos","18 a 24 anos", "25 a 34 anos",
                                                    "35 a 44 anos", "45 a 54 anos", "55 a 64 anos", "Acima de 65 anos"])
dataframe_clientes = dataframe_clientes.drop(columns=["Data Cadastro", "Hora Cadastro"])

In [ ]:
dataframe_clientes.head()

In [ ]:
dataframe_clientes.info()

### Tratativa final: removendo clientes sem CPF e exlcuindo a coluna de CPF

Aqui iremos utilizar um filtro do pandas. Repare na sintaxe. o 'tio' (~) expressa uma negação da lógica que o procede, ou seja iremos manter apenas registros onde o campo cpf não for nulo.

In [ ]:
dataframe_clientes_final = dataframe_clientes[~dataframe_clientes["cpf"].isna()]
dataframe_clientes_final = dataframe_clientes_final.drop(columns=["cpf"])

In [ ]:
dataframe_clientes_final.info()

Essa é a cara do nosso dataset de clientes finalizadas as tratativas

In [ ]:
dataframe_clientes_final.head()

Aqui usaremos uma das bibliotecas de criação de gráficos para fazer uma análise inicial da nossa base de clientes

In [ ]:
sns.countplot(x = dataframe_clientes_final["sexo"])

Percebemos que ser trata de uma base quase que totalmente composta por homens

In [ ]:
px.histogram(x=dataframe_clientes_final["idade"], nbins=100)

Aqui, notamos a predominância de um público na faixa dos 30 a 45 anos. Alem disso, temos alguns outliers de clientes cadastrados com menos de 15 anos, e outros com mais de 100 anos. Vamos limitar a base entre 16 e 80 anos

In [ ]:
dataframe_clientes_final = dataframe_clientes_final[dataframe_clientes_final["idade"].between(16, 80)]

In [ ]:
df1 = dataframe_clientes_final.groupby("regiao").agg({"id_cliente": "count"}).reset_index().sort_values(by="id_cliente", ascending=True)

fig = px.bar(df1, x="id_cliente", y="regiao", orientation='h')
fig.show()

## Tratativa da base de pedidos

### Leitura de arquivos

In [ ]:
files_pedidos = glob("/content/drive/MyDrive/Projetos/Aula pandas/Relatórios de venda anonimizados/*.xlsx")
dataframe_list_pedidos = []
for arquivo in files_pedidos:
    dataframe_temp_pedidos = pd.read_excel(io=arquivo)
    dataframe_list_pedidos.append(dataframe_temp_pedidos)

In [ ]:
dataframe_pedidos_raw = pd.concat(dataframe_list_pedidos)
dataframe_pedidos_raw.shape

### Selecionando colunas e tratativas iniciais

In [ ]:
dataframe_pedidos = dataframe_pedidos_raw[[
    "Id do cliente que realizou o pedido",
    "Id do Pedido",
    "Data da criação do pedido",
    "Hora da criação do pedido",
    "Data pagamento",
    'Valor do pedido',
    'Frete',
    'Valor total do desconto do pedido',
    "Valor total",
    "Status Resumido",
    "Status do pedido"
]]
dataframe_pedidos = dataframe_pedidos.rename(columns={
    "Id do cliente que realizou o pedido": "id_cliente",
    "Id do Pedido": "id_pedido",
    "Data da criação do pedido": "data_criacao",
    "Data pagamento": "data_pagamento",
    "Valor do pedido": "valor_pedido",
    "Frete": "valor_frete",
    "Valor total do desconto do pedido": "valor_desconto",
    "Valor total": "valor_total",
    "Status Resumido": "status_resumido",
    'Status do pedido': "status_pedido"
})

In [ ]:
dataframe_pedidos.head()

###Transformando coluna de valor em numérico

Iremos criar uma função para transformar colunas que estão como texto em número

A função de leitura de planilhas nem sempre é tão automatizada quanto ler um .csv por exemplo. Em alguns casos precisamos "parsear" os campos, ou seja, transformá-los em tipos que realmente esperamos que eles sejam. Nesse caso, estamos transformando os campos númericos (que foram lidos como string) em números

In [ ]:
def string_to_float(pandas_series):
    return pd.to_numeric(pandas_series.str.replace(".", "").str.replace(",", "."))

for col in ["valor_pedido", "valor_frete", "valor_desconto", "valor_total"]:
    dataframe_pedidos[col] = string_to_float(dataframe_pedidos[col])

Convertendo colunas de data

In [ ]:
dataframe_pedidos["data_pagamento"] = pd.to_datetime(dataframe_pedidos["data_pagamento"], format="%d/%m/%Y %H:%M:%S")
dataframe_pedidos["data_criacao"] = pd.to_datetime(dataframe_pedidos["data_criacao"] + dataframe_pedidos['Hora da criação do pedido'], format="%d/%m/%Y%H:%M:%S.%f").dt.floor('s')

In [ ]:
dataframe_pedidos.info()

In [ ]:
px.histogram(x=dataframe_pedidos["valor_total"])

Vemos que os valores dos pedidos se concentram na faixa dos 350 reais. Além disso, notamos algo de estranho, pois há uma quantidade grande de pedidos com valor entre -5 e 4,99. Vamos dar uma olhada neles

Vamos fazer um histograma dentro deste sub-dataframe

In [ ]:
px.histogram(x=dataframe_pedidos.query("valor_total < 5")["valor_total"])

Observamos que dos pedidos naquela faixa, temos que pratiamente todos eles tem o valor zerado. Geralmente as plataformas de ecommerce fazem isso em casos de troca: geram um novo pedido e dão o desconto cheio. Portanto, para pegarmos pedidos pagos, iremos filtrar além do status "Pago", apenas pedidos que possuem um valor.

### Filtrando a base

In [ ]:
# Apenas pedidos pagos com valor (excluir trocas)
dataframe_pedidos = dataframe_pedidos[dataframe_pedidos["status_resumido"] == "Pago"]
dataframe_pedidos = dataframe_pedidos[dataframe_pedidos["valor_total"] > 0]

# Excluir pedidos que não possuem ID do cliente
dataframe_pedidos = dataframe_pedidos[~dataframe_pedidos["id_cliente"].isna()]
dataframe_pedidos["id_cliente"] = dataframe_pedidos["id_cliente"].astype(int)

# Preencher data de pagamento de pedidos sem esta data com a data e criação
dataframe_pedidos["data_pagamento"] = dataframe_pedidos["data_pagamento"].fillna(dataframe_pedidos["data_criacao"])
dataframe_pedidos = dataframe_pedidos.drop(columns=["Hora da criação do pedido"])

In [ ]:
dataframe_pedidos.info()

In [ ]:
dataframe_pedidos.head()

In [ ]:
plt.hist(x = dataframe_pedidos['valor_total'], bins=100);

## Análise RFV - Cruzando os dois datasets

A análise RFV (Recência, Frequência e Valor) atribui notas para cada cliente da base, analisando quantas vezes ele comprou, o valor total que ele gastou e quanto tempo faz que ele não compra. Entende-se que os melhores clientes são aqueles que compram e gastam muito, enquanto os piores são os que compraram uma vez há muito tempo atrás. Ela é útil para direcionarmos campanhas de marketing de acordo com a estratégia que a empresa quer seguir (exemplo: fidelizar clientes fieis em potencial, consolidar fidelidade de clientes fieis, ou captar novos clientes)

In [ ]:
grouped_dataframe_pedidos = dataframe_pedidos.groupby(by="id_cliente").agg(func={
    "valor_total": "sum",
    "id_pedido": "count",
    "data_pagamento": "max",
})

grouped_dataframe_pedidos = grouped_dataframe_pedidos.rename(columns={
    "valor_total": "gmv",
    "id_pedido": "quantidade_pedidos",
    "data_pagamento": "ultimo_pedido"
}).reset_index()

grouped_dataframe_pedidos["gmv"] = grouped_dataframe_pedidos["gmv"].round(2)
grouped_dataframe_pedidos["dias_desde_ultimo_pedido"] = (datetime(2023,9,1,0,0,0) - grouped_dataframe_pedidos["ultimo_pedido"]).dt.days

dataframe_rfv = grouped_dataframe_pedidos.merge(right=dataframe_clientes_final,
               on="id_cliente",
               how="inner")

In [ ]:
px.histogram(x=dataframe_rfv["quantidade_pedidos"])

In [ ]:
px.histogram(x=dataframe_rfv["dias_desde_ultimo_pedido"])

Notamos que a maioria dos clientes comprou recentemente, mas este indicador está bem **pulverizado**. Já na quantidade de pedidos, vemos que boa parte da base comprou no máximo 2 vezes durante o período analisado

In [ ]:
# Quartil quantidade_pedidos
dataframe_rfv["quartil_quantidade_pedidos"] = pd.cut(x=dataframe_rfv["quantidade_pedidos"],
                                                bins=[0.99, 1.99, 2.99, 3.99, 5.99, 999],
                                                labels=[1, 2, 3, 4, 5],
                                                )

In [ ]:
# Quartil dias desde o último pedido
dataframe_rfv["quartil_dias_desde_ultimo_pedido"] = pd.qcut(x=dataframe_rfv["dias_desde_ultimo_pedido"],
                                                q=5,
                                                labels=[5,4,3,2,1])

# Ticket médio
dataframe_rfv["ticket_medio"] = round(dataframe_rfv["gmv"] / dataframe_rfv["quantidade_pedidos"], 2)

# Score RFV
dataframe_rfv["score_rfv"] = dataframe_rfv["quartil_dias_desde_ultimo_pedido"].astype(str) + dataframe_rfv["quartil_quantidade_pedidos"].astype(str)

# Label <> Score
seg_map = {
    r'[1-2][1-2]': 'Hibernando',
    r'[1-2][3-4]': 'Sob risco',
    r'[1-2]5': 'Imperdível',
    r'3[1-2]': 'Quase dormindo',
    r'33': 'Requer atenção',
    r'[3-4][4-5]': 'Cliente fiel',
    r'41': 'Promessa',
    r'51': 'Novos clientes',
    r'[4-5][2-3]': 'Cliente fiel em potencial',
    r'5[4-5]': 'Campeões'
}
dataframe_rfv["label_score_rfv"] = dataframe_rfv["score_rfv"].replace(seg_map, regex=True)

In [ ]:
dataframe_rfv.head()

In [ ]:
dataframe_rfv.info()

In [ ]:
dataframe_rfv.describe()

In [ ]:
grafico = px.scatter_matrix(dataframe_rfv, dimensions=['idade', 'gmv', 'dias_desde_ultimo_pedido'], color = 'label_score_rfv',
                            color_discrete_map={
        "Hibernando": "#f80301",
        "Quase dormindo": "#fc6702",
        "Sob risco": "#fb9b00",
        "Requer atenção": "#ffcc00",
        "Imperdível": "#ffff00",
        "Promessa": "#66cc00",
        "Novos clientes": "#009900",
        "Cliente fiel em potencial": "#006600",
        "Cliente fiel": "#0033ff",
        "Campeões": "#66009e"
    })
grafico.update_traces(marker=dict(size=3))
grafico.show()

In [ ]:
fig = px.scatter(
    dataframe_rfv.sort_values(by="label_score_rfv", ascending=True),
    x="quantidade_pedidos",
    y="dias_desde_ultimo_pedido",
    size="gmv",
    color="label_score_rfv",
    color_discrete_map={
        "Hibernando": "#f80301",
        "Quase dormindo": "#fc6702",
        "Sob risco": "#fb9b00",
        "Requer atenção": "#ffcc00",
        "Imperdível": "#ffff00",
        "Promessa": "#66cc00",
        "Novos clientes": "#009900",
        "Cliente fiel em potencial": "#006600",
        "Cliente fiel": "#0033ff",
        "Campeões": "#66009e"
    }
)
fig.update_layout(
    yaxis = dict(autorange="reversed"),
    autosize=False,
    width=1000,
    height=800,
)
fig.show()

## Estratégias de campanha

### 1 - Fidelizando novos clientes

Vamos descobrir qual região e faixa etária tem a maior participação de clientes nas categorias Novos Clientes e Promessa



Por região

In [ ]:
df1 = dataframe_rfv[dataframe_rfv["label_score_rfv"].isin([
    "Novos clientes", "Promessa"
])].groupby("regiao").agg({
    "id_cliente": "count"
}).reset_index()

df2 = dataframe_rfv.groupby("regiao").agg({
    "id_cliente": "count"
}).reset_index()
df = df1.merge(df2, on="regiao")

df["share_novos_regiao"] = df["id_cliente_x"] / df["id_cliente_y"]
df = df.drop(columns=["id_cliente_x", "id_cliente_y"])
df.sort_values(by="share_novos_regiao", ascending=False)

Vemos que as regiões Norte e Nordeste possuem o maior share de cliente novos ou promessas

In [ ]:
df1 = dataframe_rfv[dataframe_rfv["label_score_rfv"].isin([
    "Novos clientes", "Promessa"
])].groupby("faixa_etaria").agg({
    "id_cliente": "count"
}).reset_index()

df2 = dataframe_rfv.groupby("faixa_etaria").agg({
    "id_cliente": "count"
}).reset_index()
df = df1.merge(df2, on="faixa_etaria")
df["share_novos_fx_etaria"] = df["id_cliente_x"] / df["id_cliente_y"]
df = df.drop(columns=["id_cliente_x", "id_cliente_y"])
df.sort_values(by="share_novos_fx_etaria", ascending=False)

Aqui observamos que a faixa etária com mais clientes novos ou promessas é a dos 15 a 17 anos.

Porém... Só isso é suficiente para uma campanha de sucesso? Vamos fazer uma análise da participação nas vendas dessas categorias...

In [ ]:
df = dataframe_rfv.groupby("regiao").agg({
    "gmv": "sum"
}).reset_index()
px.pie(data_frame=df, names=df["regiao"], values=df["gmv"])

In [ ]:
df = dataframe_rfv.groupby("faixa_etaria").agg({
    "gmv": "sum"
}).reset_index()
px.pie(data_frame=df, names=df["faixa_etaria"], values=df["gmv"])

Vemos que a região Nordeste é a terceira colocada no total de vendas, representando um share de vendas até que significativo. Por outro lado a faixa etária dos 15 aos 17 anos é praticamente irrelevante no share de vendas, por isso o parâmetro da idade não deve ter um peso relevante na hora de definir um público alvo neste caso.

Portanto, uma sugestão de estratégia aos donos do négócio para conquistar novos clientes seria direcionar campanhas de marketing (propagandas ou cupons de desconto) na região Nordeste, tendo em vista que é uma região com share de vendas relevante nas vendas da empresa e ao mesmo tempo a região com maior captação de clientes novos. Caso a campanha queira ser mais direcionada, é interessante focar nas faixas etárias principais dos clientes, que no caso vimos ser dos 30 aos 45 anos.

## 2 - Consolidando clientes fiéis

Vamos descobrir qual região e faixa etária tem a maior participação de clientes nas categorias Campeões e Clientes Fieis



In [ ]:
df1 = dataframe_rfv[dataframe_rfv["label_score_rfv"].isin([
    "Campeões", "Cliente fiel"
])].groupby("regiao").agg({
    "id_cliente": "count"
}).reset_index()

df2 = dataframe_rfv.groupby("regiao").agg({
    "id_cliente": "count"
}).reset_index()
df = df1.merge(df2, on="regiao")
df["share_novos_regiao"] = df["id_cliente_x"] / df["id_cliente_y"]
df = df.drop(columns=["id_cliente_x", "id_cliente_y"])
df.sort_values(by="share_novos_regiao", ascending=False)

In [ ]:
df1 = dataframe_rfv[dataframe_rfv["label_score_rfv"].isin([
    "Campeões", "Cliente fiel"
])].groupby("faixa_etaria").agg({
    "id_cliente": "count"
}).reset_index()

df2 = dataframe_rfv.groupby("faixa_etaria").agg({
    "id_cliente": "count"
}).reset_index()
df = df1.merge(df2, on="faixa_etaria")
df["share_novos_fx_etaria"] = df["id_cliente_x"] / df["id_cliente_y"]
df = df.drop(columns=["id_cliente_x", "id_cliente_y"])
df.sort_values(by="share_novos_fx_etaria", ascending=False)

Neste caso, observamos que a região Sudeste é onde se concentram os clientes mai fiéis da empresa. Além disso, vemos que clientes entre 55 e 64 tem um share expressivo de clientes nas categorias de campeões/fiéis. A participação nas vendas desses clientes como podemos ver acima é de 3,6%, o que pode parecer não muito relevante, porém com um número absoluto de quase R$ 400.000,00 em vendas, talvez faça sentido direcionar campanhas de consolidação da fidelidade a estes clientes (um cupom de desconto direcionado a clientes fieis, ou uma pré-venda de um novo produto, por exemplo). De qualquer forma, se abrirmos o leque para a segunda faixa etária mais relevante (45 a 54), temos um share combinado que beira os 17%, tornando a campanha ainda mais relevante